In [1]:
import sys
import os
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegFileWriter
import pandas as pd
sys.path.insert(0, '../../')

import magcolloids as mgc
#pint parameter
ureg = mgc.ureg

Necessary extras

In [6]:
plt.rcParams['animation.ffmpeg_path'] = r'C:\\ffmpeg\\bin\\ffmpeg'
mplt.rcParams['agg.path.chunksize'] = 100000

def redefine_field(field,tilt_angle):
    field.fieldx = 'v_Bmag*(sin(v_freq*time*2*PI)*sin(v_theta)+cos(v_theta)*%s)'%np.sin(tilt_angle).magnitude
    field.fieldy = 'v_Bmag*(cos(v_freq*time*2*PI)*sin(v_theta))'%np.sin(tilt_angle).magnitude
    field.fieldz = 'v_Bmag*(cos(v_theta)*%s)'%np.cos(tilt_angle).magnitude
    return field

Parameters for the Simulation

In [7]:
region, initial_conditions = mgc.initial_setup(150, packing = 0.3, height = 4, radius = 1.4)

particles = mgc.particles(
    initial_conditions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.07*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

field = mgc.field(magnitude = 7.2*ureg.mT, frequency = 3.5*ureg.Hz, angle = 26.9*ureg.degrees)
field = redefine_field(field,7*ureg.deg) #default 0

world = mgc.world(particles, temperature = 300*ureg.K,
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 20*ureg.um)

sim = mgc.sim(dir_name = "C:/Users/emino/OneDrive/Desktop/MagColloids",
       timestep = 1e-4*ureg.s, framerate = 30*ureg.Hz, total_time = 60*ureg.s,
       particles = particles, world = world, field = field)

In [8]:
sim.generate_scripts()
sim.run()

CompletedProcess(args=['c:\\Users\\emino\\OneDrive\\Desktop\\MagColloids\\magcolloids-master\\magcolloids\\lammps_executables\\lmp_mingw64-native.exe', '-in', 'C:/Users/emino/OneDrive/Desktop/MagColloids\\test.lmpin'], returncode=0, stdout=b'LAMMPS (18 Jun 2019)\r\nReading data file ...\r\n  orthogonal box = (-27 -27 -2) to (27 27 2)\r\n  1 by 1 by 1 MPI processor grid\r\n  reading atoms ...\r\n  144 atoms\r\nFinding 1-2 1-3 1-4 neighbors ...\r\n  special bond factors lj:   0          0          0         \r\n  special bond factors coul: 0          0          0         \r\n  0 = max # of 1-2 neighbors\r\n  0 = max # of 1-3 neighbors\r\n  0 = max # of 1-4 neighbors\r\n  1 = max # of special neighbors\r\n  special bonds CPU = 0 secs\r\n  read_data CPU = 0.00694394 secs\r\n144 atoms in group Atoms\r\nNeighbor list info ...\r\n  update every 1 steps, delay 1 steps, check yes\r\n  max neighbors/atom: 2000, page size: 100000\r\n  master list distance cutoff = 21\r\n  ghost atom cutoff = 21\r

In [5]:
sim.load()
trj = sim.lazy_read[::10]
from IPython.display import HTML
HTML(mgc.display_animation_direct(sim,trj,speedup=1))